In [1]:
import django
django.setup()
from tree.models import Tree, Node

import os
from django.contrib.auth.models import User
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

#python manage.py shell_plus --notebook

In [2]:
def update_row_index(node):
    row = findRow(node.number)
    return [row["h_index"], row["rowNumber"]]

In [3]:
def smaller_in_row(nodeNumber):
    if nodeNumber % 2 == 1:
        #all even numbers are considered smaller
        smaller_even_numbers = row["limits"]["even_limits"][0] / 2
        smaller_odd_numbers = ( nodeNumber - row["limits"]["odd_limits"][0] )/2
    else:
        smaller_even_numbers = (row["limits"]["even_limits"][1] - nodeNumber)/2
        smaller_odd_numbers = 0 
    smaller_numbers_sum = smaller_odd_numbers + smaller_even_numbers  

def getX(nodeNumber):
    if nodeNumber == 1:
        return 0.5
    row = findRow(nodeNumber)
    
    if nodeNumber % 2 == 1:
        #all even numbers are considered smaller
        smaller_even_numbers = row["limits"]["even_limits"][0] / 2
        smaller_odd_numbers = ( nodeNumber - row["limits"]["odd_limits"][0] )/2
    else:
        smaller_even_numbers = (row["limits"]["even_limits"][1] - nodeNumber)/2
        smaller_odd_numbers = 0 
    

        
  
    smaller_numbers_sum = smaller_odd_numbers + smaller_even_numbers        
    nodes_sum = row["limits"]["even_limits"][0]
    #fairCut = 1/ (nodes_sum + 2)
    fairCut = 1/ (nodes_sum)
    #fraction of all the smaller numbers which counts in the two borders as well
    #location = fairCut* (smaller_numbers_sum + 1) + fairCut/2
    location = fairCut* (smaller_numbers_sum) + fairCut/2
    return round(location, 5)  
def merge_blobs(blobs):
    #section.sort_index(inplace = True)
    fat_node = blob[0]
    last_node = blob[1]
    fat_node.grow_to = last_node.x
    fat_node.number = last_node.number
    return fat_node

def nodes_match(first, second):
    return second.name == first.name + 1

In [71]:
import pandas as pd
import numpy as np
import sys
from tree.utilities import findRow, getY
tree = Tree.objects.first()
nodes = tree.node_set.all().values_list()
from django.contrib.auth.models import User
users = User.objects.only("username","id").values()
comprehensiveUsers = {user["id"]:user["username"] for user in users}
nodeSet = pd.DataFrame.from_records(
    nodes, 
    columns = ["id","tree_id","childrenMissing","number","userName","child1","child1Value","child2","child2Value"],
    )
nodeSet["value"]= nodeSet["child1Value"] + nodeSet["child2Value"]
nodeSet.insert(1,"changed", False)
nodeSet.insert(1,"sold", False)

nodeSet[["h_index", "row"]] = nodeSet.apply(update_row_index, axis = 1,result_type='expand')
#nodeSet["userName"] = nodeSet["userName"].apply(lambda user: comprehensiveUsers[user])
nodeSet.sort_index(inplace = True)
#nodeSet 

low_rows = nodeSet[nodeSet.row > 5].copy()
nodeSet = nodeSet[nodeSet.row < 6]
low_rows.set_index(['row','h_index'], inplace = True)
low_rows.sort_index(inplace = True)
low_rows["grow_to"] = 0 # only needed in blobs
low_rows["x"]= low_rows.apply(lambda node: getX(node.number), axis = 1)
biggest_number = low_rows.iloc[-1].number
low_rows["y"]= low_rows.apply(lambda node: getY(node.number,biggest_number), axis = 1) #wasteful
blobs = []
for row_number in low_rows.index.get_level_values(0).unique():
    
    row = low_rows.loc[row_number]#.set_index("h_index").sort_index() #add iteration for all rows
    
    row.rowd = 6 #wtf



    #row = row.drop(range(10,20)) #remove before comitting

    from collections import deque
    iterator = row.iterrows()

    start = next(iterator)[1]
    finish = None
    previous = start
    deck = deque((start,),2)
    for index, node in iterator:
        deck.append((node))
        if not nodes_match(deck[0],deck[1]): ##deck[1].name == deck[0].name+1: 
            blobs.append((start, deck[0]))
            start = deck[1]
            #print(deck[0], "blob ends here")
    blobs.append((start, deck[1]))
#blobs[0][0]
blobSet = []
for blob in blobs:
    blobSet.append(merge_blobs(blob))
blobFrame = pd.DataFrame.from_records(
    blobSet
    )
blobFrame
#row


,id,sold,changed,tree_id,childrenMissing,number,userName,child1,child1Value,child2,child2Value,value,grow_to,x,y
0,79,False,False,1,48,127,4,254,7,252,7,14,0.99219,0.00781,0.666667
1,207,False,False,1,56,255,4,510,3,508,3,6,0.99609,0.00391,0.777778
2,463,False,False,1,60,511,4,1022,1,1020,1,2,0.99805,0.00195,0.888889
3,975,False,False,1,62,959,4,2046,0,2044,0,0,0.93652,0.00098,1.000000
4,996,False,False,1,62,999,4,1965,0,1967,0,0,0.97559,0.95996,1.000000


In [72]:
nodeSet.iloc[-1].parent

AttributeError: 'Series' object has no attribute 'parent'

In [40]:
 nodeSet[nodeSet.row > 5]

,id,sold,changed,tree_id,childrenMissing,number,userName,child1,child1Value,child2,child2Value,value,h_index,row
63,64,False,False,1,48,96,4,194,7,192,7,14,15,6
64,65,False,False,1,48,98,4,198,7,196,7,14,14,6
65,66,False,False,1,48,100,4,202,7,200,7,14,13,6
66,67,False,False,1,48,102,4,206,7,204,7,14,12,6
67,68,False,False,1,48,104,4,210,7,208,7,14,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,False,False,1,62,983,4,1965,0,1967,0,0,491,9
996,997,False,False,1,62,993,4,1985,0,1987,0,0,496,9
997,998,False,False,1,62,995,4,1989,0,1991,0,0,497,9
998,999,False,False,1,62,997,4,1993,0,1995,0,0,498,9


In [94]:
rows = row.iterrows()

next(rows)[1].name

0

In [73]:
iterator = peekable(row.iterrows())
iterator.peek()

(0,
 id                    79
 sold               False
 changed            False
 tree_id                1
 childrenMissing       48
 number               126
 userName               4
 child1               254
 child1Value            7
 child2               252
 child2Value            7
 value                 14
 row                    6
 grow_to                0
 x                    0.5
 Name: 0, dtype: object)

In [53]:
iterator = row.iterrows()
next(iterator)[1].name

0

In [37]:
iterator = row.iterrows()

next(iterator)
iterator.send("gay")
 


(1,
 id                    78
 sold               False
 changed            False
 tree_id                1
 childrenMissing       48
 number               124
 userName               4
 child1               250
 child1Value            7
 child2               248
 child2Value            7
 value                 14
 row                    6
 grow_to                0
 x                   0.25
 Name: 1, dtype: object)

In [43]:
from more_itertools import spy, peekable
iterator = row.iterrows()


(current, nexte), iterator = spy(iterator, 2)

(current, nexte)



((0,
  id                    79
  sold               False
  changed            False
  tree_id                1
  childrenMissing       48
  number               126
  userName               4
  child1               254
  child1Value            7
  child2               252
  child2Value            7
  value                 14
  row                    6
  grow_to                0
  x                    0.5
  Name: 0, dtype: object),
 (1,
  id                    78
  sold               False
  changed            False
  tree_id                1
  childrenMissing       48
  number               124
  userName               4
  child1               250
  child1Value            7
  child2               248
  child2Value            7
  value                 14
  row                    6
  grow_to                0
  x                   0.25
  Name: 1, dtype: object))

In [14]:
mario

2

In [145]:
iterator = row.iterrows()
start_node = next(iterator)[1]
eat_until = None
for h_index, next_node in iterator:   
    if next_node.userName ==  start_node.userName:
        eat_until = h_index
    else:
        start_node.grow_to = getX(row[eat_until].number)
row

,id,sold,changed,tree_id,childrenMissing,number,userName,child1,child1Value,child2,child2Value,value,row,grow_to,x
h_index,,,,,,,,,,,,,,,
0,79,False,False,1,48,126,4,254,7,252,7,14,6,0,0.50000
1,78,False,False,1,48,124,4,250,7,248,7,14,6,0,0.25000
2,77,False,False,1,48,122,4,246,7,244,7,14,6,0,0.37500
3,76,False,False,1,48,120,4,242,7,240,7,14,6,0,0.12500
4,75,False,False,1,48,118,4,238,7,236,7,14,6,0,0.43750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,123,False,False,1,48,119,4,237,7,239,7,14,6,0,0.89062
60,124,False,False,1,56,121,4,241,3,243,3,6,6,0,0.92188
61,125,False,False,1,51,123,4,245,4,247,7,11,6,0,0.95312


In [34]:
def get_row_index(number):
    row = findRow(number)
    
def insert_row_index(func):
    def wrapper(nodeNumber):

        
        row = func(nodeNumber)
        if nodeNumber == 1:
            smaller_even_numbers = 0
            smaller_odd_numbers = 0 

        if nodeNumber % 2 == 1:
            #all even numbers are considered smaller
            smaller_even_numbers = row["limits"]["even_limits"][0] / 2
            smaller_odd_numbers = ( nodeNumber - row["limits"]["odd_limits"][0] )/2
        else:
            smaller_even_numbers = (row["limits"]["even_limits"][1] - nodeNumber)/2
            smaller_odd_numbers = 0 
    

        
  
        row["h_index"] = int(smaller_odd_numbers + smaller_even_numbers ) 
        return row
    return wrapper 
@insert_row_index
def findRow(nodeNumber, startNode =1):
    #potential infinite loop D:
    if nodeNumber == startNode:
        return {"rowNumber":0, "limits": {"even_limits": None, "odd_limits": None}, "size":1,"index" : 0}

    if startNode == 1:

        rowNumber  = 1
        limits = [2, 3]
        nodes_in_row = 2
        even_limits = [2,2]
        odd_limits = [3,3]
        while not (nodeNumber >limits[0]/2 and nodeNumber <= limits[1]):
            limits = [limits[0] *2 +2, limits[1] * 2 +1]
            even_limits = [int(limits [0]/2) + 1, limits[0]]
            odd_limits = [int(limits[0]/2) +2, limits[1]]       
            rowNumber +=1
            nodes_in_row = int(limits[1]- limits[0]/2)
        #return rowNumber, [even_limits, odd_limits], 
        row = {
            "rowNumber":rowNumber, 
            "limits": {
                "even_limits": even_limits, 
                "odd_limits": odd_limits},
            "size":limits[1] - limits[0]+1,
            "index": nodeNumber - 1
        }
        return row
    if startNode % 2 == 1:
        rowNumber  = 1
        index = 0
        limits = [startNode*2 -1, startNode *2 +1]
        
        while not (nodeNumber >=limits[0] and nodeNumber <= limits[1]):
            rowNumber +=1
            index += (limits[0]+1)/2 -1            
            limits[0] = limits[0] * 2 -1
            limits[1] = limits [1] *2 +1
        middle = limits[0] + (limits[1]-limits[0])/2        
        if nodeNumber < middle:
            index += middle - nodeNumber            
        if nodeNumber > middle:
            index +=  nodeNumber - middle +1
        row = {
            "rowNumber":rowNumber, 
            "limits": limits,
            "size":(limits[1] - limits[0])/2+1,
            "index": index
        }
        return row
    if startNode % 2 == 0:       
        rowNumber  = 1
        index = 0
        limits = [startNode*2 +2, startNode *2]
        while not (nodeNumber >=limits[1] and nodeNumber <= limits[0]):
            rowNumber +=1
            index += limits[1]/2            
            limits[0] = limits[0] * 2 +2
            limits[1] = limits [1] *2 
        middle = limits[0] - ((limits[0] - limits[1])/2)          
        if nodeNumber < middle:
            index += middle - nodeNumber +1         
        if nodeNumber > middle:
            index +=  nodeNumber - middle
        row = {
            "rowNumber":rowNumber, 
            "limits": limits,
            "size":(limits[1] - limits[0])/2+1,
            "index": index
        }
        return row

findRow(10)

{'rowNumber': 3,
 'limits': {'even_limits': [8, 14], 'odd_limits': [9, 15]},
 'size': 2,
 'index': 9,
 'h_index': 2}

In [27]:
findRow(10)

{'rowNumber': 3,
 'limits': {'even_limits': [8, 14], 'odd_limits': [9, 15]},
 'size': 2,
 'index': 9,
 'h_index': 2.0}

In [38]:
from tree.utilities import findRow
findRow(14)

{'rowNumber': 3,
 'limits': {'even_limits': [8, 14], 'odd_limits': [9, 15]},
 'size': 2,
 'index': 13}

0.21094

In [38]:
from tree.utilities import nodeGenerator
try:     
    id = Node.objects.all().order_by("-id").first().id + 1 
except:
    id = 1

from tree.utilities import childOf, nodeGenerator, getIndex

def getFirstFreeNode(startNode, id, tree_id, user, nodeSet):
    for node in nodeGenerator(startNode):
        if node not in nodeSet.index:
            generator = nodeGenerator(node)
            next(generator)
            #### flawed if there are multiple trees
            row = {
                'id': id,
                'sold':False,
                'changed':True,
                'tree_id': tree_id, # should be a varaiable
                'childrenMissing': 62,
                'userName': user,##fix this
                'child1': next(generator),
                'child1Value': 0,
                'child2':next(generator),
                'child2Value': 0,
                'value': 299,    
                }
            nodeSet.loc[node] = row
            return nodeSet.loc[node]
id+=1
getFirstFreeNode(startNode = 6,id= id, tree_id = 1, user = 1, nodeSet = nodeSet)
    

id                 15769
sold               False
changed             True
tree_id                1
childrenMissing       62
userName               1
child1             28678
child1Value            0
child2             28676
child2Value            0
value                299
Name: 14338, dtype: object

In [34]:
nodeSet

,id,sold,changed,tree_id,childrenMissing,userName,child1,child1Value,child2,child2Value,value
number,,,,,,,,,,,
1,2,False,False,1,0,3,2,5151,3,10110,15261
2,3,False,False,1,0,3,6,2047,4,3103,5150
3,34,False,False,1,0,3,5,3102,7,7007,10109
4,4,False,False,1,0,3,10,1055,8,2047,3102
5,35,False,False,1,0,3,9,2047,11,1054,3101
...,...,...,...,...,...,...,...,...,...,...,...
4146157,15769,False,True,1,62,1,8292313,0,8292315,0,299
4146163,15769,False,True,1,62,1,8292325,0,8292327,0,299
4146155,15769,False,True,1,62,1,8292309,0,8292311,0,299


In [54]:

result = nodeSet.loc[nodeSet.tree_id == 1].index.min()   #.reset_index().min("number")
pd.isnull(result)
max(result,1)

nan

In [21]:
from tree.utilities import findParent, getX, getY
nodeSet["parent"] = nodeSet.apply(lambda node: findParent(node.name), axis = 1)
biggest_number = nodeSet.iloc[-1].name
nodeSet["x"]= nodeSet.apply(lambda node: getX(node.name), axis = 1)
nodeSet["y"]= nodeSet.apply(lambda node: getY(node.name,biggest_number), axis = 1)
nodeSet

,id,sold,changed,tree_id,childrenMissing,userName,child1,child1Value,child2,child2Value,value,parent,x,y
number,,,,,,,,,,,,,,
1,2,False,False,1,58,mario,2,0,3,4,4,NaN,0.5000,0.111111
3,3,False,False,1,59,mario,5,2,7,1,3,1.0,0.7500,0.333333
5,4,False,False,1,61,mario,9,1,11,0,1,3.0,0.6250,0.555556
7,5,False,False,1,62,mario,13,0,15,0,0,3.0,0.8750,0.777778
9,6,False,False,1,62,mario,17,0,19,0,0,5.0,0.5625,1.000000


In [19]:
nodeSet.iloc[-1].name

9

In [178]:
oldSet = Node.objects.filter(pk__in = [332])                            
oldSet.delete()   

(0, {})

get the __ID__

In [3]:
id = nodeSet.id.max() + 1
#gotta test this
id = id if not np.isnan(id) else 1

__Start Node__

In [338]:
smallest = nodeSet.loc[mask1, "childrenMissing"].min()
allSmalls = nodeSet.loc[nodeSet.childrenMissing == smallest,:].copy()
allSmalls["index"] = allSmalls.apply(lambda node: getIndex(node.name, 1), axis=1)
allSmalls.sort_values("index", inplace = True)
payNode = allSmalls.iloc[0]
for node in nodeGenerator(payNode.name):
    if node not in nodeSet.index:
        generator = nodeGenerator(node)
        next(generator)
        id +=1 #### flawed if there are multiple trees
        row = {
            'id': id,
            'sold':False,
            'changed':True,
            'tree_id': 1, # should be a varaiable
            'childrenMissing': 62,
            'user_id': 1,##fix this
            'child1': next(generator),
            'child1Value': 0,
            'child2':next(generator),
            'child2Value': 0,
            'value': 299,    
            }
        nodeSet.loc[node] = row
        amount -=1
        startNode = node
        break
startNode
#find first free child

79

find easiest node to pay __findNodeToPay__

In [5]:
from tree.utilities import childOf, nodeGenerator, getIndex
#randomly picked start index
startNode = 1 # start node can only be picked but not created
nodeSet.sort_values(by = "childrenMissing", inplace =True)
mask1 = nodeSet.childrenMissing != 0
mask2 = nodeSet.apply(lambda node : childOf(startNode, node.name) , axis=1)

#mask for the user aswell
smallest = nodeSet.loc[mask1 & mask2, "childrenMissing"].min()
mask3 = nodeSet.childrenMissing == smallest
allSmalls = nodeSet.loc[mask1 & mask2 & mask3,:].copy()

allSmalls["index"] = allSmalls.apply(lambda node: getIndex(node.name, startNode), axis=1)
allSmalls.sort_values("index", inplace = True)
payNode = allSmalls.iloc[0]
payNode


id                     2
sold               False
changed            False
tree_id                1
childrenMissing       24
userName           mario
child1                 2
child1Value           20
child2                 3
child2Value           18
value                 38
index                  0
Name: 1, dtype: object

__Make Children Table__

In [6]:
# this amount is the number of nodes the user buys
targetValue = 31
# ideally each child would be eat least 31
#using copy just for testing
#children = nodeSet.loc[[payNode.child1, payNode.child2]].copy()
children = pd.DataFrame([
    [payNode.child1, payNode.child1Value],
    [payNode.child2, payNode.child2Value]],
    columns = ["number","value"])
#if child does not exist it will throw error

children["index"] = children.apply(lambda kid: getIndex(kid.number, payNode.name), axis = 1)
children["pay"] = 0
#children.sort_values("index")
children.iloc[1,-3] = 3
children.sort_values(by=["value", "index"], inplace = True) #index not necessary?
children

,number,value,index,pay
1,3,3,2,0
0,2,20,1,0


__distribute an amount__

figure out how many nodes go under each side

In [7]:
amount = 100

#equalize children

childDifference = children.iloc[1].value - children.iloc[0].value
#this might be pointless because the smaller one is always at the top
valueMissing = max(targetValue - children.iloc[0].value, 0)
pay = min(childDifference,valueMissing, amount)

amount -= pay
children.iloc[0,-1]+= pay
children.iloc[0,-3]+= pay

children.sort_values(by=["index"], inplace = True)
# attempt to reach target

valueMissing = max(targetValue - children.iloc[0].value, 0) *2
pay = min(amount, valueMissing)
amount -= pay
half = int(pay/2)
children.iloc[[0,1],[-1, -3]] += half

leftover = pay - (half*2)
children.iloc[[0],[-1, -3]] +=leftover #leftover is 1 or 0




children

,number,value,index,pay
0,2,31,1,11
1,3,31,2,28


__pick the right node numbers__

In [34]:
generator = nodeGenerator(payNode.name)
child = 0
childToggle ={0:1,1:0}
childrenStacks = [[],[]]
pay = children.pay.sum()
while pay > 0 :  
    potentialNode = next(generator)    
    if not potentialNode in nodeSet.index:        
        if len(childrenStacks[child]) < children.iloc[child,-1]:
            childrenStacks[child].append(potentialNode)
            pay -= 1 
    child = childToggle[child]   
print(childrenStacks)
childrenStacks = childrenStacks[0] + childrenStacks[1] #merge them as it doesnt matter later on in code

[[447, 449, 451, 453, 455, 457, 459, 461, 463, 465, 467], [662, 664, 666, 668, 670, 672, 674, 676, 678, 680, 682, 684, 686, 688, 690, 692, 694, 696, 698, 700, 702, 704, 706, 708, 710, 712, 714, 716]]


__append rows__

In [117]:

for node in range(716, 100000000000):
    generator = nodeGenerator(node)
    next(generator) # skip the first one
    id +=1 #### flawed if there are multiple trees
    row = {
        'id': id,
        'sold':False,
        'changed':True,
        'tree_id': 1, # should be a varaiable
        'childrenMissing': 62,
        'user_id': 1,
        'child1': next(generator),
        'child1Value': 0,
        'child2':next(generator),
        'child2Value': 0,
        'value': 299,    
        }
    nodeSet.loc[node] = row
    
nodeSet
    


KeyboardInterrupt: 

In [118]:
nodeSet

,id,sold,changed,tree_id,childrenMissing,userName,child1,child1Value,child2,child2Value,value
number,,,,,,,,,,,
1,2,True,False,1,0,mario,2,328,3,145,38
2,3,False,False,1,25,mario,6,6,4,320,18
3,13,True,False,1,0,mario,5,32,7,111,16
4,4,True,False,1,0,mario,10,262,8,56,10
5,14,False,False,1,32,mario,9,28,11,2,8
...,...,...,...,...,...,...,...,...,...,...,...
10535,10472,False,True,1,62,NaN,21069,0,21071,0,299
10536,10473,False,True,1,62,NaN,21074,0,21072,0,299
10537,10474,False,True,1,62,NaN,21073,0,21075,0,299


__Update Parents__

In [55]:
from tree.utilities import parentGenerator
def updateParent(parentNode, child): # nodePair["child"]
        
        if parentNode.child1 == child:
            parentNode.child1Value +=1                                         
        if parentNode.child2 ==  child:
            parentNode.child2Value +=1

        #update childrenMissing value
        #fix this to use apply()
        #nodeSet.loc[parentNode,"old] = nodeSet.loc[parentNode].apply(function)
        soldBefore = parentNode.childrenMissing == 0            
        child1Value = parentNode.child1Value           
        child1Value = child1Value if child1Value <32 else 31
        child2Value = parentNode.child2Value
        child2Value = child2Value if child2Value <32 else 31            
        parentNode.childrenMissing = 62 - child1Value - child2Value
        soldNow = parentNode.childrenMissing == 0
        if (soldNow and not soldBefore):# if the node just got sold
            parentNode.sold = True
            print(parentNode.name)
      
        return parentNode
fuck = 0    
for node in childrenStacks:    
        for nodePair in parentGenerator(node):
            fuck += 1
            #print(nodeSet.loc[nodePair["parent"],["child1Value","child2Value","childrenMissing","sold"]])
            nodeSet.loc[[nodePair["parent"]]] = nodeSet.loc[[nodePair["parent"]]].apply(updateParent,args= (nodePair["child"],),axis = 1, result_type= "expand")
            #nodeSet.loc[[nodePair["parent"]]].update(updateParent,args= (nodePair["child"],),axis = 1, result_type= "expand")
print(fuck)

3
340


In [116]:
fuck = 0  
for nodePair in parentGenerator(447):
            fuck += 1
            #print(nodeSet.loc[nodePair["parent"],["child1Value","child2Value","childrenMissing","sold"]])
            nodeSet.loc[[nodePair["parent"]]] = nodeSet.loc[[nodePair["parent"]]].apply(
                updateParent,args= (nodePair["child"],),axis = 1, result_type= "expand")
            #nodeSet.loc[[nodePair["parent"]]].update(
            #updateParent,args= (nodePair["child"],),axis = 1, result_type= "expand")
print(fuck)

8


In [135]:
def updateParents(node, origin):    
    if parentOf(node.child1,origin):
        node.child1Value +=1                                         
    else:
        node.child2Value +=1
    child1Value = min(node.child1Value,31)
    child2Value = min(node.child2Value,31)
    node.childrenMissing = 62 - child1Value - child2Value    
    return node
for child in newNodes:
    ancestors = [node["parent"] for node in parentGenerator(child)]
    
    nodeSet.loc[ancestors] = nodeSet.loc[ancestors].apply(afunction, args = [child], axis = 1)
nodeSet
#nodeSet.loc[3]

[1239, 619, 309, 155, 77, 39, 19, 9, 5, 3, 1]


KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [124]:
newNodes = nodeSet.iloc[-8000: -1, 0].to_list()

In [128]:
newNodes

[2477,
 2478,
 2479,
 2480,
 2481,
 2482,
 2483,
 2484,
 2485,
 2486,
 2487,
 2488,
 2489,
 2490,
 2491,
 2492,
 2493,
 2494,
 2495,
 2496,
 2497,
 2498,
 2499,
 2500,
 2501,
 2502,
 2503,
 2504,
 2505,
 2506,
 2507,
 2508,
 2509,
 2510,
 2511,
 2512,
 2513,
 2514,
 2515,
 2516,
 2517,
 2518,
 2519,
 2520,
 2521,
 2522,
 2523,
 2524,
 2525,
 2526,
 2527,
 2528,
 2529,
 2530,
 2531,
 2532,
 2533,
 2534,
 2535,
 2536,
 2537,
 2538,
 2539,
 2540,
 2541,
 2542,
 2543,
 2544,
 2545,
 2546,
 2547,
 2548,
 2549,
 2550,
 2551,
 2552,
 2553,
 2554,
 2555,
 2556,
 2557,
 2558,
 2559,
 2560,
 2561,
 2562,
 2563,
 2564,
 2565,
 2566,
 2567,
 2568,
 2569,
 2570,
 2571,
 2572,
 2573,
 2574,
 2575,
 2576,
 2577,
 2578,
 2579,
 2580,
 2581,
 2582,
 2583,
 2584,
 2585,
 2586,
 2587,
 2588,
 2589,
 2590,
 2591,
 2592,
 2593,
 2594,
 2595,
 2596,
 2597,
 2598,
 2599,
 2600,
 2601,
 2602,
 2603,
 2604,
 2605,
 2606,
 2607,
 2608,
 2609,
 2610,
 2611,
 2612,
 2613,
 2614,
 2615,
 2616,
 2617,
 2618,
 2619,

In [88]:
def parentOf(parent,node):
    if parent == node:
        return True
    for node in parentGenerator(node):
        if node["parent"] == parent:
            return True
    return False

parentOf(445,447)

False

In [139]:
ancestors

[1239, 619, 309, 155, 77, 39, 19, 9, 5, 3, 1]

In [141]:
nodeSet.loc[619]

KeyError: 619

In [562]:
def updateParent(parentNode, child): # nodePair["child"]
    #informs the parents their child is sold
    if parentNode.child1 == child:
        parentNode.child1Value +=1                                         
    if parentNode.child2 ==  child:
        parentNode.child2Value +=1
    soldBefore = parentNode.childrenMissing == 0            
    child1Value = parentNode.child1Value  
    #print(parentNode.child1Value)
    child1Value = child1Value if child1Value <32 else 31
    child2Value = parentNode.child2Value
    child2Value = child2Value if child2Value <32 else 31            
    parentNode.childrenMissing = 62 - child1Value - child2Value
    soldNow = parentNode.childrenMissing == 0
    if (soldNow and not soldBefore):# if the node just got sold
        parentNode.sold = True
        print(parentNode.name)   
    #print(parentNode[["child1Value","child2Value","childrenMissing","sold"]])
    return parentNode[["child1Value","child2Value","childrenMissing","sold"]].to_list()

for node in childrenStacks:    
        for nodePair in parentGenerator(node):
            
            #print(nodeSet.loc[nodePair["parent"],["child1Value","child2Value","childrenMissing","sold"]])
            parentNode = nodeSet.loc[nodePair["parent"]].copy() 
            #very very slow
            nodeSet.loc[nodePair["parent"],["child1Value","child2Value","childrenMissing","sold"]]=updateParent(parentNode, nodePair["child"])

            

5


In [564]:
for node in childrenStacks:    
        for nodePair in parentGenerator(node):
            parentNode = nodeSet.loc[nodePair["parent"]]    
                       #informs the parents their child is sold
            if parentNode.child1 == child:
                parentNode.child1Value +=1                                         
            if parentNode.child2 ==  child:
                parentNode.child2Value +=1
            soldBefore = parentNode.childrenMissing == 0            
            child1Value = parentNode.child1Value  
            #print(parentNode.child1Value)
            child1Value = child1Value if child1Value <32 else 31
            child2Value = parentNode.child2Value
            child2Value = child2Value if child2Value <32 else 31            
            parentNode.childrenMissing = 62 - child1Value - child2Value
            soldNow = parentNode.childrenMissing == 0
            if (soldNow and not soldBefore):# if the node just got sold
                parentNode.sold = True
                print(parentNode.name)   
            #print(parentNode[["child1Value","child2Value","childrenMissing","sold"]])
        

        

C:\Users\Robert\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [546]:
nodeSet.loc[nodeSet.sold == True]

,id,sold,changed,tree_id,childrenMissing,user_id,child1,child1Value,child2,child2Value,value
number,,,,,,,,,,,
5,6,True,False,1,0,3,9,302,11,110,117
33,34,True,False,1,0,3,65,140,67,155,0
67,547,True,True,1,0,1,133,75,135,75,299
133,548,True,True,1,0,1,265,35,267,35,299
135,549,True,True,1,0,1,269,35,271,35,299
65,578,True,True,1,0,1,129,60,131,75,299
131,579,True,True,1,0,1,261,35,263,35,299


In [536]:
nodeSet

,id,sold,changed,tree_id,childrenMissing,user_id,child1,child1Value,child2,child2Value,value
number,,,,,,,,,,,
1,2,False,False,1,0,3,2,63,3,236,299
2,3,False,False,1,0,3,6,31,4,31,62
3,4,False,False,1,0,3,5,118,7,117,235
4,5,False,False,1,32,3,10,15,8,15,30
5,6,False,False,1,24,3,9,7,11,110,117
...,...,...,...,...,...,...,...,...,...,...,...
1039,601,False,True,1,62,1,2077,0,2079,0,299
1037,602,False,True,1,62,1,2073,0,2075,0,299
1035,603,False,True,1,62,1,2069,0,2071,0,299


In [414]:
type(nodeSet.loc[1,["id", "child1"]])

pandas.core.series.Series

In [419]:
def someFunction(node):
    print (node)

In [424]:
mario = nodeSet.loc[1]
mario

id                               2
sold                         False
changed                      False
tree_id                          1
childrenMissing                  0
user_id                          3
child1             696969669696969
child1Value                     63
child2                           3
child2Value                   1120
value                          299
Name: 1, dtype: object

In [411]:
nodeSet.loc[1].id

2

In [193]:
mario = nodeSet.loc[
    nodeSet.changed==True,
    [
        "id",
        "tree_id",
        "childrenMissing",
        "user_id",
        "child1",
        "child1Value",
        "child2",
        "child2Value"
    ]].reset_index().copy()
#mario.apply(lambda node: print(node))
#mario.drop(mario[:5].index, inplace = True)
#mario.iloc[:5]
#mario.to_dict(orient="record" )[0]
mario.to_dict(orient="record" )[0] 

{'number': 79,
 'id': 311,
 'tree_id': 1,
 'childrenMissing': 62,
 'user_id': 1,
 'child1': 157,
 'child1Value': 0,
 'child2': 159,
 'child2Value': 0}

In [241]:
sold = nodeSet.loc[nodeSet.sold == True,["sold","user_id"]]
sold

,sold,user_id
number,,
5,True,3
79,True,1
159,True,1
317,True,1
157,True,1
315,True,1


In [246]:
sold.groupby("user_id").count()

,sold
user_id,
1,5
3,1


In [248]:
sold.groupby("user_id").count().reset_index().to_dict(orient="record" )

[{'user_id': 1, 'sold': 5}, {'user_id': 3, 'sold': 1}]

In [5]:
nodeSet.loc[1,"sold"] = True
nodeSet.loc[2,"sold"] = True
nodeSet.loc[63,"sold"] = True
nodeSet

,id,sold,changed,tree_id,childrenMissing,userName,child1,child1Value,child2,child2Value,value
number,,,,,,,,,,,
1,2,True,False,1,0,3,2,31,3,130,161
2,3,True,False,1,32,3,6,15,4,15,30
3,34,False,False,1,16,3,5,15,7,114,129
4,4,False,False,1,48,3,10,7,8,7,14
5,35,False,False,1,48,3,9,7,11,7,14
...,...,...,...,...,...,...,...,...,...,...,...
4059,162,False,False,1,62,1,8117,0,8119,0,0
4061,161,False,False,1,62,1,8121,0,8123,0,0
4063,160,False,False,1,62,1,8125,0,8127,0,0


In [11]:
sold = nodeSet.loc[nodeSet.sold == True,["sold","userName"]].groupby("userName").count().copy()
sold = sold.reset_index().to_dict(orient="record" )

sold = [{item["userName"]:item["sold"] for item in sold}]
sold

C:\Users\Robert\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\frame.py:1482: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{1: 1, 3: 2}]

In [3]:
dictionary = [{'userName': 1, 'sold': 2}, {'userName': 3, 'sold': 1}]

In [611]:
nodeSet.loc[:,["userName","child1","child1Value","child2","child2Value","childrenMissing"]].reset_index().to_json(orient="records")

'[{"number":1,"userName":"mario3","child1":2,"child1Value":63,"child2":3,"child2Value":236,"childrenMissing":0},{"number":2,"userName":"mario3","child1":6,"child1Value":31,"child2":4,"child2Value":31,"childrenMissing":0},{"number":3,"userName":"mario3","child1":5,"child1Value":118,"child2":7,"child2Value":117,"childrenMissing":0},{"number":4,"userName":"mario3","child1":10,"child1Value":15,"child2":8,"child2Value":15,"childrenMissing":32},{"number":5,"userName":"mario3","child1":9,"child1Value":7,"child2":11,"child2Value":110,"childrenMissing":24},{"number":6,"userName":"mario3","child1":14,"child1Value":15,"child2":12,"child2Value":15,"childrenMissing":32},{"number":7,"userName":"mario3","child1":13,"child1Value":109,"child2":15,"child2Value":7,"childrenMissing":24},{"number":8,"userName":"mario3","child1":18,"child1Value":7,"child2":16,"child2Value":7,"childrenMissing":48},{"number":9,"userName":"mario3","child1":17,"child1Value":3,"child2":19,"child2Value":3,"childrenMissing":56},{"

In [587]:
from django.contrib.auth.models import User
users = User.objects.only("username","id").values()
comprehensiveUsers = {user["id"]:user["username"] for user in users}
comprehensiveUsers

{1: 'mario', 2: 'lksjdf', 3: 'mario3'}

In [7]:
import json
data = []
data.append(json.loads(tree.json_string))
json.dumps(data)

'[[{"number": 1, "userName": "mario", "child1": 2, "child1Value": 5, "child2": 3, "child2Value": "", "childrenMissing": 53, "parent": null, "x": 0.5, "y": 0.0454545455}, {"number": 2, "userName": "mario", "child1": 6, "child1Value": 0, "child2": 4, "child2Value": 4, "childrenMissing": 58, "parent": 1.0, "x": 0.25, "y": 0.0909090909}, {"number": 3, "userName": "mario", "child1": 5, "child1Value": 2, "child2": 7, "child2Value": 1, "childrenMissing": 59, "parent": 1.0, "x": 0.75, "y": 0.1363636364}, {"number": 4, "userName": "mario", "child1": 10, "child1Value": 2, "child2": 8, "child2Value": 1, "childrenMissing": 59, "parent": 2.0, "x": 0.375, "y": 0.1818181818}, {"number": 5, "userName": "mario", "child1": 9, "child1Value": 1, "child2": 11, "child2Value": 0, "childrenMissing": 61, "parent": 3.0, "x": 0.625, "y": 0.2272727273}, {"number": 7, "userName": "mario", "child1": 13, "child1Value": 0, "child2": 15, "child2Value": 0, "childrenMissing": 62, "parent": 3.0, "x": 0.875, "y": 0.318181

In [3]:
from tree.utilities import getIndex
getIndex(12,6)

13.0


2.0